# My implentation of the pagerank

We import libraries

In [9]:
import networkx as nx
import numpy as np
import scipy.sparse as sp
import sys


In [10]:

try:
    import cugraph
    import cudf
    HAS_GPU = True
    print("trovata GPU")
except ImportError:
    HAS_GPU = False
    print("NO GPU")

NO GPU


In [11]:
def build_index_map(G):
    nodes = list(G.nodes())
    node_to_idx = {node: i for i,node in enumerate(nodes)}
    return nodes, node_to_idx

In [12]:
def build_transition_matrix(G, node_to_idx):
    n= len(G)
    row_ind = []
    col_ind = []
    data = []
    dangling_nodes = []
    
    for node in G.nodes():
        u = node_to_idx[node]
        neighbors = list(G.neighbors(node))
        
        if not neighbors:
            #dangling node --> no out-links 
            dangling_nodes.append(u)
        else:
            weight = 1.0 / len(neighbors)
            for neighbor in neighbors:
                v= node_to_idx[neighbor]
                row_ind.append(u)
                col_ind.append(v)
                data.append(weight)
                
    #compressed Sparse row matrix, we use sparse because is more compact
    M= sp.csr_matrix((data,(row_ind, col_ind)),shape=(n,n))
    return M, dangling_nodes

In [13]:
def power_iteration(M, num_nodes, dangling_nodes, alpha=0.85, tol=1.0e-6, max_iter=200):
    #Formula: r_new = alpha * M.T * r_old + (1 - alpha) / N + dangling_correction
    # 1. Initialize PageRank vector uniformly (1/N)
    x = np.ones(num_nodes) / num_nodes
    
    # 2. Pre-calculate the teleportation probability (random jump factor)
    # The probability of jumping to any random node is (1-alpha) / N
    teleport_prob = (1 - alpha) / num_nodes
    
    # Check for convergence
    for i in range(max_iter):
        x_prev = x.copy()
        
        # --- The "Random Walk" Step ---
        
        # Calculate sum of PR mass from dangling nodes
        # If a walker is at a dangling node, they jump to a random node.
        dangling_sum = sum(x_prev[d] for d in dangling_nodes)
        dangling_val = alpha * (dangling_sum / num_nodes)
        
        # Matrix multiplication: Push mass along edges
        # M.T accounts for incoming links (transpose of transition matrix)
        x = alpha * (M.T @ x_prev)
        
        # Add the teleportation factor + dangling node redistribution
        x += teleport_prob + dangling_val
        
        # Check convergence (L1 norm of difference)
        err = np.abs(x - x_prev).sum()
        if err < tol:
            print(f"Converged after {i+1} iterations.")
            return x

    print("Max iterations reached without full convergence.")
    return x

In [14]:
def calculate_pagerank(G, alpha = 0.85):
    
    n= len(G)
    
    nodes, node_to_idx = build_index_map(G)
    
    # 3. Build Transition Matrix (The "Graph Logic")
    print("Building transition matrix...")
    M, dangling_nodes = build_transition_matrix(G, node_to_idx)
    
    # 4. Run Solver (The "Math Logic")
    print("Running PageRank power iteration...")
    scores = power_iteration(M, n, dangling_nodes, alpha=alpha)
    
    # 5. Format Output
    return dict(zip(nodes, scores))

MAYBE BEttER IMPLEMENTATION

In [15]:
from scipy.sparse.linalg import gmres

def get_pagerank_linear_system(G, alpha=0.85, tol=1e-6):
    """
    Calcola PageRank risolvendo il sistema lineare (I - alpha * M^T) * x = v.
    
    A differenza del metodo iterativo (Power Iteration), questo approccio vede il PageRank
    come la soluzione di un sistema di equazioni lineari. Utilizza il metodo GMRES
    (Generalized Minimal Residual), che appartiene alla famiglia dei metodi del 
    sottospazio di Krylov.
    
    È spesso più robusto e veloce della Power Iteration standard, specialmente quando
    il damping factor (alpha) è vicino a 1 (dove la Power Iteration converge lentamente).
    
    Args:
        G (nx.DiGraph): Il grafo NetworkX (deve essere diretto).
        alpha (float): Damping factor (default 0.85).
        tol (float): Tolleranza per la convergenza del solver (usata come rtol).
        
    Returns:
        dict: Dizionario {nodo_id: punteggio_pagerank}
    """
    print(f"🔹 Inizio calcolo PageRank (Metodo GMRES) su {G.number_of_nodes()} nodi...")
    
    # Lista ordinata dei nodi per mantenere la corrispondenza con gli indici della matrice
    nodelist = list(G)
    N = len(nodelist)
    
    if N == 0:
        return {}
    
    # 1. Creazione della Matrice di Adiacenza Sparsa
    # M_ij = 1 se esiste arco i -> j, altrimenti 0
    # Usiamo il formato CSR (Compressed Sparse Row) per efficienza aritmetica
    try:
        M = nx.to_scipy_sparse_array(G, nodelist=nodelist, dtype=float, format="csr")
    except AttributeError:
        # Fallback per versioni vecchie di NetworkX
        M = nx.to_scipy_sparse_matrix(G, nodelist=nodelist, dtype=float, format="csr")
    
    # 2. Normalizzazione delle Righe (Matrice Stocastica P)
    # P_ij = 1 / out_degree(i)  se esiste arco i -> j
    
    # Calcolo gradi uscenti (somma delle righe)
    degrees = np.array(M.sum(axis=1)).flatten()
    
    # Identifica i nodi "dangling" (senza uscite, grado 0)
    # Questi nodi non distribuiscono massa nella matrice P, ma la disperdono globalmente
    dangling_nodes = np.where(degrees == 0)[0]
    
    # Calcolo inverso dei gradi per la normalizzazione (evitando divisione per 0)
    # Se grado è 0, mettiamo 1 (tanto poi non verrà usato perché la riga è vuota)
    with np.errstate(divide='ignore'):
        inv_degrees = 1.0 / np.maximum(degrees, 1)
        
    # Creazione matrice diagonale D^-1
    D_inv = sp.diags(inv_degrees)
    
    # Matrice di transizione sparsa P = D^-1 * M
    P = D_inv.dot(M)
    
    # 3. Costruzione del Sistema Lineare Ax = b
    # L'equazione di PageRank è: x = alpha * P^T * x + (1-alpha)/N * e + (alpha/N * x_dangling_sum) * e
    # Riorganizzando come sistema lineare (semplificato):
    # (I - alpha * P^T) * x = v_teleport
    
    # Operatore A = I - alpha * P^T
    # Nota: Usiamo la trasposta P.T perché PageRank è l'autovettore sinistro (o destro di P^T)
    I = sp.eye(N, format='csr')
    A = I - alpha * P.T
    
    # Vettore termine noto b (vettore di teletrasporto uniforme)
    # Rappresenta la probabilità (1-alpha) che l'utente salti a caso
    # b_i = (1 - alpha) / N
    b = np.repeat((1.0 - alpha) / N, N)
    
    # 4. Risoluzione con GMRES
    print("   Risoluzione del sistema lineare sparso Ax = b...")
    
    # GMRES trova x tale che Ax = b minimizzando il residuo
    # x0 (guess iniziale) è uniforme 1/N
    x0 = np.repeat(1.0/N, N)
    
    # --- MODIFICA QUI: tol -> rtol ---
    # Scipy recente usa 'rtol' per relative tolerance. 
    # 'atol' è absolute tolerance (opzionale, qui lo impostiamo uguale a rtol per sicurezza)
    try:
        x, exitCode = gmres(A, b, x0=x0, rtol=tol, atol=tol)
    except TypeError:
        # Fallback per versioni vecchie di Scipy che usavano 'tol'
        x, exitCode = gmres(A, b, x0=x0, tol=tol, atol=tol)
    
    if exitCode != 0:
        print(f"⚠️  Attenzione: GMRES non ha convergiuto completamente (codice {exitCode})")
        if exitCode > 0:
            print(f"    Convergenza mancata dopo {exitCode} iterazioni.")
    else:
        print("✅  GMRES ha convergiuto con successo.")
    
    # 5. Normalizzazione Finale
    # Il risultato di GMRES potrebbe non sommare esattamente a 1 a causa dell'approssimazione
    # dei dangling nodes nel termine noto. Normalizziamo L1 per garantire che sia una distribuzione di probabilità.
    x = x / np.sum(x)
    
    # 6. Conversione in Dizionario
    pagerank_scores = dict(zip(nodelist, x))
    
    return pagerank_scores